In [3]:
import pickle
import numpy as np
import numpy as np
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model, load_model
from keras.layers.recurrent import GRU,LSTM
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
import keras.callbacks
import itertools


In [2]:
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args

    # the 2 is critical here since the first couple outputs of the RNN
    # tend to be garbage:
    y_pred = y_pred[:, 2:, :]
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
with open("sequences.txt", "r") as f:
    seq_labels = pickle.load(f)
with open("sequences_features.txt", "r") as f2:
    seq_features = pickle.load(f2)


IOError: [Errno 2] No such file or directory: 'sequences.txt'

In [57]:
def get_data(a,b):
    list1 = []
    list2 = []
    for i in range(a, b):
        list1.append(seq_features["Sequence_" + str(i) + ".png"])

        list2.append(seq_labels["Sequence_" + str(i) + ".png"])

    arr1 = np.asarray(list1, dtype=np.float32)
    arr2 = np.asarray(list2, dtype=np.int64)
    return arr1, arr2,np.ones((b-a,1))*30,np.ones((b-a,1))*5

In [2]:
model=load_model('first_try_LSTM_CTC.h5', custom_objects = {'<lambda>': lambda y_true, y_pred: y_pred})
#model3 = Model(inputs=model.input,outputs=model.get_layer('softmax').output)
#print seq_labels['Sequence_90500.png']

Tensor("softmax/truediv:0", shape=(?, 32, 73), dtype=float32)


In [8]:
model3 = Model(inputs=model.get_layer('the_input').input,outputs=model.get_layer('softmax').output)

In [9]:
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
the_input (InputLayer)          (None, 32, 128)      0                                            
__________________________________________________________________________________________________
lstm1 (LSTM)                    (None, 32, 128)      131584      the_input[0][0]                  
__________________________________________________________________________________________________
lstm1b (LSTM)                   (None, 32, 128)      131584      the_input[0][0]                  
__________________________________________________________________________________________________
add_1 (Add)                     (None, 32, 128)      0           lstm1[0][0]                      
                                                                 lstm1b[0][0]                     
__________

In [50]:
seq_labels['Sequence_90601.png']

[29, 49, 39, 10, 21]

In [51]:
X=seq_features['Sequence_90601.png']
X=np.expand_dims(X,axis=0)

In [85]:

S=model3.predict(X)

In [86]:
S.shape

(1, 32, 73)

In [84]:
import intertools
def decode_batch(out):
    ret = []
    for j in range(out.shape[0]):
        out_best = list(np.argmax(out[j, 2:], 1))
        out_best = [k for k, g in itertools.groupby(out_best)]
        outstr = []
        for c in out_best:
           
                outstr.append(c)
        ret.append(outstr)
    return ret

In [55]:
decode_batch(S)

['29 72 49 72 39 72 10 72 21 ']

In [88]:
S,labels,_,_=get_data(90000,100000)

In [89]:
S.shape

(10000, 32, 128)

In [90]:
labels.shape

(10000, 5)

In [91]:
V=model3.predict(S,batch_size=10000)

In [92]:
V.shape

(10000, 32, 73)

In [93]:
pred= decode_batch(V)

In [98]:
error=0
for i in range(10000):
    t=0
    for s in range (len(pred[i])):
        if t==5:
            break
        if pred[i][s]==72:
            continue
        print labels[i][t]
        print pred[i][s]
        if pred[i][s]!=labels[i][t]:
            error+=1
        t+=1
print error
            
    


62
62
11
11
43
43
24
24
54
54
30
30
18
18
1
1
37
37
6
6
62
62
5
5
3
3
2
2
70
70
67
67
65
65
8
8
5
5
26
26
41
41
12
12
18
18
37
37
54
54
56
56
63
63
35
35
66
66
70
70
22
22
52
52
32
32
59
59
67
67
29
29
35
35
32
32
55
55
20
20
46
46
24
24
20
20
55
55
33
33
14
14
8
8
42
42
37
37
32
32
26
26
2
2
67
67
67
67
28
28
41
41
6
6
34
34
2
2
63
63
52
52
3
3
13
13
52
52
5
5
26
26
17
17
0
0
18
18
0
0
58
58
40
40
2
2
51
51
32
32
35
35
1
1
20
20
31
31
55
55
16
16
14
14
39
39
69
69
43
43
1
1
12
12
59
59
35
35
44
44
26
26
7
7
36
36
70
70
56
56
51
51
47
47
45
45
24
24
70
70
21
21
60
60
6
6
8
8
57
57
30
30
57
57
17
17
54
54
2
2
24
24
11
11
48
48
66
66
48
48
14
14
58
58
44
44
30
30
60
60
71
71
33
33
24
24
31
31
59
59
8
8
39
39
39
39
40
40
10
10
2
2
38
38
17
17
18
18
16
16
6
6
24
24
0
0
20
20
10
10
45
45
61
61
70
70
61
61
10
10
8
8
42
42
2
2
37
37
37
37
37
37
23
23
38
38
24
24
10
10
34
34
39
39
37
37
6
6
54
54
44
44
48
48
42
42
45
45
16
16
67
67
8
8
8
8
11
11
50
50
63
63
65
65
19
19
28
28
65
65
50
50
0
0
25

40
40
52
52
36
36
37
37
27
27
39
39
43
43
26
26
39
39
10
10
50
50
0
0
56
56
62
62
47
47
23
23
46
46
55
55
66
66
55
55
21
21
66
66
34
34
42
42
7
7
1
1
63
63
40
40
54
54
63
63
1
1
68
68
1
1
6
6
34
34
0
0
61
61
65
65
29
29
46
46
20
20
34
34
25
25
47
47
69
69
38
38
5
5
24
24
54
54
39
39
37
37
67
67
66
66
33
33
10
10
7
7
21
21
17
17
14
14
63
63
6
6
27
27
65
65
15
15
23
23
10
10
10
10
29
29
44
44
45
45
37
37
43
43
0
0
23
23
12
12
26
26
60
60
55
55
2
2
64
64
44
44
39
39
38
38
65
65
40
40
24
24
25
25
18
18
54
54
4
4
71
71
57
57
12
12
14
14
54
54
62
62
47
47
50
1
35
35
61
61
53
53
47
47
21
21
0
0
5
5
10
10
52
52
48
48
49
49
52
52
67
67
55
55
15
15
8
8
69
69
41
41
6
6
37
37
15
15
24
24
0
0
15
15
13
13
43
43
26
26
67
67
3
3
66
66
71
71
57
57
47
47
54
54
53
53
3
3
45
45
70
70
19
19
70
70
62
62
36
36
52
52
44
44
4
4
1
1
31
31
4
4
24
24
18
18
50
50
20
20
67
67
4
4
32
32
67
67
36
36
1
1
28
28
24
24
69
69
10
10
64
64
57
57
59
59
38
38
49
49
60
60
38
38
26
26
66
66
68
68
6
6
10
10
70
70
50
50
22
22
4
4

7
59
59
62
62
57
57
4
4
38
38
39
39
12
12
38
38
38
38
36
36
58
58
28
28
51
51
39
39
53
53
60
60
2
2
63
63
52
52
35
35
6
6
51
51
7
7
61
61
2
2
47
47
42
42
58
58
69
69
31
31
60
60
41
41
43
43
23
23
43
43
57
57
56
56
0
0
18
18
58
58
28
28
18
18
7
7
48
48
42
42
19
19
14
14
51
51
20
20
39
39
2
2
7
7
30
30
23
23
40
40
33
33
28
28
54
54
14
14
57
57
39
39
3
3
42
42
68
68
45
45
38
38
55
55
8
8
60
60
49
49
0
0
40
40
38
38
13
13
55
55
33
33
44
44
49
49
46
46
63
63
27
27
33
33
66
66
61
61
9
9
16
16
18
18
26
26
65
65
10
10
10
10
23
23
51
51
32
32
36
36
23
23
51
51
43
43
21
21
20
20
37
37
56
56
0
0
37
37
11
11
64
64
68
68
0
0
41
41
26
26
31
31
39
39
39
39
66
66
57
57
41
41
69
69
29
29
26
26
22
22
61
61
25
25
33
33
66
66
70
70
2
2
44
44
27
27
16
16
7
7
2
2
58
58
37
37
38
38
69
69
61
61
38
38
46
46
9
9
19
19
66
66
1
1
42
42
66
66
43
43
7
7
46
46
67
67
47
47
59
59
35
35
49
49
9
9
33
33
61
61
15
15
21
21
34
34
1
1
28
28
3
3
12
12
51
51
0
0
23
23
14
14
1
1
40
40
48
48
9
9
31
31
22
22
63
63
69
69
15
15
61

52
52
32
32
67
67
60
60
1
1
50
50
13
13
31
31
48
48
46
46
26
26
53
53
10
10
50
50
61
61
65
65
68
68
37
37
44
44
52
52
2
2
63
63
3
3
20
20
39
39
57
57
0
0
63
63
65
65
65
65
11
11
30
50
8
8
61
61
0
0
24
24
16
16
5
5
15
15
6
6
43
43
22
22
14
14
68
68
2
2
70
70
56
56
49
49
24
24
24
24
36
36
69
69
8
8
70
70
44
44
19
19
29
29
19
19
45
45
69
69
65
65
61
61
4
4
51
51
69
69
12
12
41
41
64
64
41
41
28
28
6
6
41
41
63
63
57
57
11
11
50
50
68
68
22
22
15
15
62
62
61
61
17
17
20
20
44
44
49
49
17
17
10
10
13
13
52
52
9
9
19
19
36
36
7
7
55
55
19
19
1
1
7
7
71
71
47
47
28
28
45
45
0
0
7
7
49
49
11
11
18
18
42
42
25
25
27
27
38
38
9
9
49
49
31
31
15
15
9
9
55
55
69
69
63
63
64
64
43
43
0
0
26
26
60
60
57
57
30
30
69
69
42
43
67
67
4
4
10
10
43
43
19
19
36
36
69
69
47
47
41
41
21
21
14
14
29
29
12
12
28
28
42
42
18
18
29
29
55
55
60
60
17
17
33
33
51
51
19
19
25
25
20
20
48
48
51
51
42
42
60
60
25
25
30
30
33
33
64
64
61
61
24
24
31
31
61
61
17
17
44
44
30
30
46
46
27
27
8
8
13
13
51
51
13
13
3
3
59
5

11
11
48
48
1
1
25
25
45
45
27
27
64
64
65
65
53
53
3
3
33
33
25
25
25
25
36
36
16
16
3
3
54
54
1
1
67
67
27
27
5
5
20
20
61
61
23
23
67
67
65
65
34
34
52
52
7
7
50
50
54
54
33
33
8
8
18
18
13
13
4
4
14
14
0
0
6
6
58
58
27
27
70
70
48
48
60
60
24
24
33
33
27
27
51
51
42
42
11
11
31
31
28
28
54
54
70
70
23
23
35
35
70
70
0
0
63
63
31
31
31
31
64
64
66
66
53
53
58
58
68
68
46
46
71
71
29
29
27
27
44
44
3
3
54
54
32
32
9
9
13
13
55
55
61
61
44
44
65
65
23
23
9
9
11
11
11
11
15
15
49
49
71
71
22
22
61
61
58
58
61
61
70
70
45
45
35
35
44
44
30
29
8
8
59
59
29
29
15
15
34
34
66
66
10
10
0
0
38
38
60
60
1
1
62
62
16
16
39
39
43
43
17
17
9
9
53
53
54
54
68
68
39
39
40
40
37
37
0
0
54
54
44
44
59
59
70
70
26
26
5
5
20
20
71
71
9
9
36
36
18
18
64
64
43
43
20
20
45
45
5
5
23
23
10
10
31
31
57
57
63
63
48
48
1
1
60
60
15
15
0
0
67
67
33
33
70
70
31
31
29
29
58
58
6
6
5
5
31
31
19
19
31
31
9
9
2
2
20
20
11
11
25
25
9
9
14
14
47
47
8
8
2
2
40
40
56
56
63
63
50
50
18
18
16
16
47
47
19
19
34
35
70
70


...

64
64
58
58
34
34
69
69
39
39
3
3
70
70
8
8
30
30
36
36
55
55
38
38
68
68
15
15
41
41
42
42
30
30
63
63
10
10
61
61
55
55
32
32
5
5
70
70
29
29
43
43
18
18
55
55
66
66
9
9
17
17
44
44
65
65
4
4
45
45
21
21
9
9
48
48
3
3
29
29
50
50
1
1
4
4
24
24
3
3
30
30
60
60
34
34
37
37
14
14
65
65
29
29
65
65
33
33
65
65
26
26
43
43
53
53
31
31
31
31
13
13
67
67
46
46
60
60
23
23
59
59
70
70
19
19
44
44
24
24
28
28
62
62
5
5
5
5
50
50
56
56
71
71
1
1
51
51
32
32
31
31
28
28
50
50
25
25
19
19
15
15
67
67
63
63
6
6
1
1
58
58
63
63
34
34
15
15
37
37
4
4
25
25
22
22
44
44
47
47
34
34
60
60
65
65
9
9
4
4
22
22
45
45
6
6
66
66
39
39
14
14
28
28
65
65
2
2
44
44
67
67
18
18
64
64
61
61
27
27
51
51
52
52
26
26
1
1
46
46
31
31
51
51
3
3
53
53
15
15
58
58
3
3
12
12
3
3
26
26
6
6
12
12
44
44
47
47
45
45
24
24
17
17
38
38
69
69
52
52
38
38
0
0
31
31
54
54
13
13
13
13
52
52
29
29
62
62
41
41
64
64
21
21
12
12
38
38
9
9
8
8
36
36
31
31
35
35
2
2
2
2
63
63
15
15
56
56
5
5
53
53
16
16
50
50
64
64
9
9
52
52
33
33
1


42
42
25
25
45
45
54
54
37
37
55
55
18
18
7
7
2
2
17
17
15
15
25
25
9
9
65
65
27
27
43
43
70
70
51
51
55
55
64
64
29
29
37
37
42
42
14
14
66
66
65
65
19
19
57
57
5
5
45
45
57
57
4
4
22
22
62
62
48
48
15
15
30
30
16
16
17
17
52
52
50
50
1
1
10
10
31
31
24
24
57
57
10
10
20
20
26
26
38
38
56
56
29
29
30
30
47
47
39
39
24
24
34
34
55
55
31
31
49
49
61
61
2
2
42
42
4
4
23
23
35
35
67
67
2
2
15
15
9
9
63
63
7
7
44
44
57
57
15
15
42
42
60
60
51
51
8
8
13
13
16
16
32
32
33
33
38
38
49
49
30
30
8
8
2
2
63
63
63
63
34
34
38
38
29
29
33
33
10
10
3
3
31
31
12
12
17
17
55
55
62
62
9
9
2
2
54
54
55
55
70
70
6
6
71
71
57
11
4
4
46
46
61
61
58
60
36
36
7
7
42
42
24
24
37
37
27
27
39
39
52
52
25
25
5
5
67
67
6
6
10
10
68
68
0
0
11
11
35
35
39
39
51
51
19
19
65
65
54
54
9
9
6
6
4
4
28
28
33
33
49
49
58
58
60
60
42
42
68
68
15
15
2
2
68
68
55
55
55
55
54
54
2
2
16
16
71
71
34
34
38
38
43
43
24
24
48
48
66
66
0
0
30
30
6
6
43
43
3
3
44
44
19
19
33
33
14
14
14
14
50
50
12
12
62
62
7
7
1
1
54
54
1
1
68
68


1
52
52
14
14
51
51
64
64
20
20
25
25
59
59
35
35
67
67
58
58
23
23
28
28
43
43
23
23
42
42
71
71
29
29
18
18
27
27
30
30
7
7
57
57
35
35
65
65
22
22
13
13
18
18
70
70
68
68
7
7
13
13
23
23
53
53
24
24
58
58
61
61
70
70
42
42
44
44
52
52
63
63
6
6
19
19
38
38
54
54
44
44
43
43
64
64
48
48
2
2
49
49
51
51
13
13
13
13
22
22
42
42
22
22
14
14
56
56
38
38
58
58
44
44
31
31
53
53
38
38
28
28
42
42
12
12
19
19
49
49
9
9
44
44
50
50
38
38
24
24
58
58
62
62
51
51
59
59
15
15
2
2
0
0
49
49
61
61
54
54
0
0
21
21
23
23
38
38
50
50
61
61
27
27
71
71
31
31
70
70
27
27
21
21
27
27
53
53
41
41
18
18
64
64
53
53
21
21
58
58
16
16
55
55
32
32
67
67
44
44
7
7
36
36
29
29
1
1
47
47
48
48
25
25
5
5
4
4
23
23
28
28
55
54
47
47
54
54
24
24
17
17
64
64
39
39
45
45
30
30
35
35
32
32
12
12
1
1
47
47
48
49
44
44
48
48
20
20
31
31
56
56
64
64
36
36
8
8
67
67
12
12
51
51
15
15
42
42
44
44
50
50
13
13
57
57
63
63
29
29
20
20
60
60
32
32
43
43
4
4
60
60
28
28
48
48
40
40
51
51
28
28
4
4
25
25
46
46
10
10
0
0
64
64


31
26
26
53
53
53
53
42
42
48
48
36
36
37
37
24
24
59
59
51
51
65
65
2
2
27
27
63
63
52
52
60
60
31
31
68
68
45
45
71
71
64
64
36
36
68
68
60
60
71
71
24
24
39
39
60
60
7
7
14
14
62
62
54
54
20
20
26
26
30
30
17
17
34
34
2
2
23
23
59
59
46
46
60
60
13
13
59
59
25
25
15
15
32
32
57
57
58
58
69
69
15
15
49
49
45
45
24
24
66
66
71
71
45
45
13
13
38
38
31
31
57
57
70
70
14
14
9
9
43
43
63
63
16
16
21
21
47
47
63
63
48
48
56
56
43
43
46
46
33
33
40
40
16
16
27
27
37
37
59
0
3
3
13
13
36
36
63
63
55
54
44
44
56
56
1
1
15
15
19
19
37
37
66
66
45
45
46
46
37
37
7
7
63
63
18
18
50
50
28
28
61
61
40
40
51
51
63
63
59
59
14
14
18
18
12
12
20
20
38
38
8
8
6
6
22
22
28
28
2
2
31
31
27
68
40
40
1
1
46
46
2
2
16
16
26
26
7
7
20
20
20
20
8
8
56
56
46
46
9
9
59
59
18
18
52
52
66
66
60
60
56
56
25
25
39
39
68
68
43
43
31
31
10
10
53
53
12
12
20
20
31
31
40
40
4
4
30
30
55
55
36
36
16
16
24
24
64
64
60
60
55
55
59
59
17
17
28
28
12
12
6
6
30
30
37
37
39
39
45
45
55
55
0
0
7
7
67
67
71
71
12
12
30
30
64
6

47
47
43
43
52
52
11
11
52
52
52
52
47
47
20
20
11
11
5
5
32
32
33
33
10
10
18
18
42
42
40
40
48
48
63
63
27
27
35
35
26
26
44
44
7
7
18
18
19
19
25
25
3
3
59
59
26
26
40
40
56
56
13
13
44
44
35
35
21
21
0
0
36
36
13
13
11
11
50
50
17
17
32
32
13
13
60
60
52
52
32
32
68
68
20
20
39
39
14
14
0
0
14
14
18
18
38
38
5
5
37
37
70
70
59
59
49
49
37
37
32
32
59
59
13
13
21
21
62
62
48
48
16
16
46
46
53
53
0
0
53
53
39
39
26
26
27
27
64
64
61
61
56
56
52
52
39
39
34
34
58
58
8
8
62
62
27
27
50
50
19
19
23
23
5
5
36
36
44
44
46
46
16
16
57
57
6
6
29
29
48
48
6
6
54
54
21
21
49
49
32
32
19
19
2
2
39
39
70
70
40
40
71
71
11
11
11
11
70
70
68
68
47
47
49
49
6
6
56
56
47
47
50
50
0
0
70
70
37
37
17
17
50
50
9
9
4
4
66
66
9
9
49
49
46
46
42
42
34
34
65
65
41
41
20
20
22
22
38
38
33
33
22
22
33
33
26
26
6
6
16
16
0
0
20
20
24
24
40
40
30
30
17
17
71
71
19
19
13
13
64
64
33
33
71
71
56
56
59
59
40
40
21
21
22
22
47
47
56
56
16
16
49
49
21
21
6
6
1
1
35
35
35
35
51
51
37
37
60
60
14
14
53
53
0
0
48
48


In [100]:
504.0/(10000*5)

0.01008